# Libraries

In [11]:
from pandas import read_excel, DataFrame
from sklearn.preprocessing import OneHotEncoder


# Load Data

In [12]:
data_file_path = r'data (1).xlsx'

meta_data_df = read_excel(data_file_path,sheet_name=0)
meta_data_df.head()

,Columns,Description,Variable
0,appointmentId,Unique id,Independent
1,inspectionStartTime,inspection date,Independent
2,year,registration year,Independent
3,month,registration month,Independent
4,engineTransmission_battery_value,condition ok: yes\nelse : no,Independent


In [ ]:
engine_df = read_excel(data_file_path,sheet_name=1)
engine_df.head()

# Data Preprocessing & EDA

In [4]:
df_dups = engine_df.drop(['appointmentId','inspectionStartTime'],axis=1)
df_dups = df_dups.drop_duplicates()


In [ ]:
# There are 6 duplicate rows present in the data
df_dups.reset_index()

,index,year,month,engineTransmission_battery_value,engineTransmission_battery_cc_value_0,engineTransmission_battery_cc_value_1,engineTransmission_battery_cc_value_2,engineTransmission_battery_cc_value_3,engineTransmission_battery_cc_value_4,engineTransmission_engineoilLevelDipstick_value,...,engineTransmission_gearShifting_cc_value_1,engineTransmission_gearShifting_cc_value_2,engineTransmission_comments_value_0,engineTransmission_comments_value_1,engineTransmission_comments_value_2,engineTransmission_comments_value_3,engineTransmission_comments_value_4,fuel_type,odometer_reading,rating_engineTransmission
0,0,2008,8,No,Weak,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,Pickup Low,Starter Motor / Solenoid malfunctioning,NaN,NaN,NaN,Petrol + CNG,124600,3.0
1,1,2007,5,Yes,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petrol + CNG,78108,4.0
2,2,2012,5,Yes,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diesel,93348,3.5
3,3,2013,1,Yes,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petrol + CNG,48233,3.5
4,4,2011,7,Yes,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petrol + CNG,74368,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26296,26302,2013,3,Yes,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,Electrical wiring Damaged,NaN,NaN,NaN,NaN,Petrol + CNG,64532,4.0
26297,26303,2007,8,No,Weak,NaN,NaN,NaN,NaN,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petrol,88051,4.0
26298,26304,2004,7,Yes,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petrol,22656,3.5
26299,26305,2010,12,Yes,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diesel,126015,4.5


In [ ]:
engine_df.describe()

,inspectionStartTime,year,month,engineTransmission_engineOil_cc_value_9,engineTransmission_engine_cc_value_10,odometer_reading,rating_engineTransmission
count,26307,26307.000000,26307.000000,0.0,0.0,26307.000000,26307.000000
mean,2019-02-24 04:17:07.174554624,2010.856578,5.462006,NaN,NaN,76460.143764,3.624663
min,2019-01-02 10:02:34,1989.000000,1.000000,NaN,NaN,1.000000,0.500000
25%,2019-01-29 13:54:15.500000,2008.000000,2.000000,NaN,NaN,46396.000000,3.500000
50%,2019-02-24 12:57:51,2011.000000,5.000000,NaN,NaN,72013.000000,4.000000
75%,2019-03-23 10:11:12.500000,2014.000000,9.000000,NaN,NaN,98289.500000,4.000000
max,2019-04-15 12:47:00,2019.000000,12.000000,NaN,NaN,999999.000000,5.000000
std,NaN,3.766234,3.583866,NaN,NaN,46762.524489,0.847645


In [7]:
# check for dtypes
df_dtypes = engine_df.dtypes
df_dtypes

appointmentId                                  object
inspectionStartTime                    datetime64[ns]
year                                            int64
month                                           int64
engineTransmission_battery_value               object
                                            ...      
engineTransmission_comments_value_3            object
engineTransmission_comments_value_4            object
fuel_type                                      object
odometer_reading                                int64
rating_engineTransmission                     float64
Length: 73, dtype: object

In [ ]:
df_filtr = engine_df.drop(['appointmentId'],axis=1)
df_filtr

,inspectionStartTime,year,month,engineTransmission_battery_value,engineTransmission_battery_cc_value_0,engineTransmission_battery_cc_value_1,engineTransmission_battery_cc_value_2,engineTransmission_battery_cc_value_3,engineTransmission_battery_cc_value_4,engineTransmission_engineoilLevelDipstick_value,...,engineTransmission_gearShifting_cc_value_1,engineTransmission_gearShifting_cc_value_2,engineTransmission_comments_value_0,engineTransmission_comments_value_1,engineTransmission_comments_value_2,engineTransmission_comments_value_3,engineTransmission_comments_value_4,fuel_type,odometer_reading,rating_engineTransmission
0,2019-02-03 15:43:48,2008,8,No,Weak,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,Pickup Low,Starter Motor / Solenoid malfunctioning,NaN,NaN,NaN,Petrol + CNG,124600,3.0
1,2019-01-16 13:02:12,2007,5,Yes,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petrol + CNG,78108,4.0
2,2019-02-09 13:31:04,2012,5,Yes,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diesel,93348,3.5
3,2019-01-18 11:02:53,2013,1,Yes,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petrol + CNG,48233,3.5
4,2019-01-27 12:12:06,2011,7,Yes,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petrol + CNG,74368,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26302,2019-03-10 13:08:03,2013,3,Yes,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,Electrical wiring Damaged,NaN,NaN,NaN,NaN,Petrol + CNG,64532,4.0
26303,2019-04-12 13:59:02,2007,8,No,Weak,NaN,NaN,NaN,NaN,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petrol,88051,4.0
26304,2019-02-28 10:42:53,2004,7,Yes,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petrol,22656,3.5
26305,2019-04-02 12:21:19,2010,12,Yes,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diesel,126015,4.5


## Data Encoding


In [10]:

ohe = OneHotEncoder(sparse=False)
df = DataFrame(ohe.fit_transform(df[['Color']]), columns=ohe.get_feature_names_out(['Color'])))

SyntaxError: unmatched ')' (203462459.py, line 2)

## Handling missing values

In [ ]:
#check for nan values and its percentage 
missing_perc = (df_filtr.isna().sum()/ df_filtr.shape[0])*100
missing_perc

inspectionStartTime                       0.000000
year                                      0.000000
month                                     0.000000
engineTransmission_battery_value          0.000000
engineTransmission_battery_cc_value_0    86.931235
                                           ...    
engineTransmission_comments_value_3      99.775725
engineTransmission_comments_value_4      99.946782
fuel_type                                 0.000000
odometer_reading                          0.000000
rating_engineTransmission                 0.000000
Length: 72, dtype: float64

In [ ]:
# filter the columns with more than 50% of nan/missing values
# and remove these columns as they don't hold any significant info
## HERE 50% is INDUSTRY standards
missing_perc_80 = missing_perc[missing_perc > 80]
missing_perc_80

engineTransmission_battery_cc_value_0                    86.931235
engineTransmission_battery_cc_value_1                    98.365454
engineTransmission_battery_cc_value_2                    99.726309
engineTransmission_battery_cc_value_3                    99.939180
engineTransmission_battery_cc_value_4                    99.984795
engineTransmission_engineOilLevelDipstick_cc_value_0     98.437678
engineTransmission_engineOil_cc_value_3                  85.775649
engineTransmission_engineOil_cc_value_4                  93.264150
engineTransmission_engineOil_cc_value_5                  97.685027
engineTransmission_engineOil_cc_value_6                  99.540046
engineTransmission_engineOil_cc_value_7                  99.958186
engineTransmission_engineOil_cc_value_8                  99.992397
engineTransmission_engineOil_cc_value_9                 100.000000
engineTransmission_engine_cc_value_1                     80.674345
engineTransmission_engine_cc_value_2                     90.97

In [ ]:
# missing_perc_80.index.tolist()

In [ ]:
df_filtr = df_filtr.drop( missing_perc_80.index.tolist(), axis=1)
df_filtr

,inspectionStartTime,year,month,engineTransmission_battery_value,engineTransmission_engineoilLevelDipstick_value,engineTransmission_engineOil,engineTransmission_engineOil_cc_value_0,engineTransmission_engineOil_cc_value_1,engineTransmission_engineOil_cc_value_2,engineTransmission_engine_value,...,engineTransmission_exhaustSmoke_value,engineTransmission_exhaustSmoke_cc_value_0,engineTransmission_engineBlowByBackCompression_value,engineTransmission_engineBlowByBackCompression_cc_value_0,engineTransmission_clutch_value,engineTransmission_clutch_cc_value_0,engineTransmission_gearShifting_value,fuel_type,odometer_reading,rating_engineTransmission
0,2019-02-03 15:43:48,2008,8,No,Yes,No,Leaking,Dirty,Leakage from Tappet Cover,No,...,No,White,No,Back Compression Exist,No,Spongy,Yes,Petrol + CNG,124600,3.0
1,2019-01-16 13:02:12,2007,5,Yes,Yes,No,Leaking,Dirty,Leakage from Tappet Cover,No,...,No,Noise from silencer assembly,Yes,NO Blow-by,Yes,NaN,Yes,Petrol + CNG,78108,4.0
2,2019-02-09 13:31:04,2012,5,Yes,Yes,No,Dirty,NaN,NaN,Yes,...,No,Black,Yes,Perm. Blow By on idle,No,Hard,Yes,Diesel,93348,3.5
3,2019-01-18 11:02:53,2013,1,Yes,Yes,No,Dirty,NaN,NaN,Yes,...,Yes,NaN,Yes,NO Blow-by,No,Hard,Yes,Petrol + CNG,48233,3.5
4,2019-01-27 12:12:06,2011,7,Yes,Yes,No,Leaking,Leakage from Tappet Cover,NaN,No,...,No,White,Yes,NO Blow-by,No,Spongy,Yes,Petrol + CNG,74368,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26302,2019-03-10 13:08:03,2013,3,Yes,Yes,No,Leaking,Dirty,Leakage from Tappet Cover,Yes,...,Yes,NaN,Yes,NO Blow-by,Yes,NaN,Yes,Petrol + CNG,64532,4.0
26303,2019-04-12 13:59:02,2007,8,No,No,No,Dirty,NaN,NaN,No,...,Yes,NaN,No,Back Compression Exist,No,Hard,Yes,Petrol,88051,4.0
26304,2019-02-28 10:42:53,2004,7,Yes,Yes,No,Leaking,Dirty,Leakage from Tappet Cover,No,...,Yes,NaN,Yes,NO Blow-by,No,Hard,Yes,Petrol,22656,3.5
26305,2019-04-02 12:21:19,2010,12,Yes,Yes,No,Dirty,Leakage from Sump/chamber,NaN,Yes,...,No,Black,Yes,NO Blow-by,Yes,NaN,Yes,Diesel,126015,4.5


In [ ]:
# check for unique values i.e variance of data

variances = df_filtr.nunique()
variances

inspectionStartTime                                          26203
year                                                            28
month                                                           12
engineTransmission_battery_value                                 2
engineTransmission_engineoilLevelDipstick_value                  2
engineTransmission_engineOil                                     2
engineTransmission_engineOil_cc_value_0                          9
engineTransmission_engineOil_cc_value_1                          8
engineTransmission_engineOil_cc_value_2                          7
engineTransmission_engine_value                                  2
engineTransmission_engine_cc_value_0                            12
engineTransmission_coolant_value                                 2
engineTransmission_coolant_cc_value_0                            4
engineTransmission_engineMounting_value                          2
engineTransmission_engineMounting_cc_value_0                  

In [ ]:
# df_dups = df_filtr.drop(['inspectionStartTime'],axis=1)
# df_dups = df_dups.drop_duplicates()
# df_dups.reset_index()

## Outlier Detection

## Feature Selection

In [ ]:
# checking for correlation between indpendent and dependent varaibles
correlation = DataFrame(df_filtr.corr())

ValueError: could not convert string to float: 'No'

# Model Selection

# Model Training and Evaluation

# Model Fine Tuning

# End